In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Uploading CPS data

In [2]:
cps = pd.read_csv('cps_test.csv')

# Best Linear Predictor

In [3]:
ols1 = smf.ols(formula='log_wage ~ educ',data=cps).fit(cov_type='HC1')

print(ols1.summary())

                            OLS Regression Results                            
Dep. Variable:               log_wage   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     2441.
Date:                Tue, 08 Aug 2023   Prob (F-statistic):               0.00
Time:                        11:56:55   Log-Likelihood:                -29890.
No. Observations:               22715   AIC:                         5.978e+04
Df Residuals:                   22713   BIC:                         5.980e+04
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.5711      0.034    254.650      0.0

In [4]:
beta1 = ols1._results.params[1]

In [5]:
ols2 = smf.ols(formula='log_wage ~ educ+age',data=cps).fit(cov_type='HC1')

print(ols2.summary())

                            OLS Regression Results                            
Dep. Variable:               log_wage   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     2684.
Date:                Tue, 08 Aug 2023   Prob (F-statistic):               0.00
Time:                        11:56:55   Log-Likelihood:                -28323.
No. Observations:               22715   AIC:                         5.665e+04
Df Residuals:                   22712   BIC:                         5.668e+04
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.4145      0.039    187.891      0.0

In [6]:
beta2_0 = ols2._results.params[0]
beta2_1 = ols2._results.params[1]
beta2_2 = ols2._results.params[2]

# Creating Interval Data

In [7]:
def createIntervalData(df, Y, thresholds):
    # The function accepts a dataframe, df, and make interval data from the Y (string) variable. 
    # It adds to the dataframe the lower and upper values for Y (based on the thresholds) and the covariates.
    
    thresholds = np.array(thresholds)

    idx = [sum(t <= y for t in thresholds)-1 for y in df[Y]]
    df[Y+'_l'] = thresholds[idx]
    df[Y+'_u'] = thresholds[np.array(idx)+1]
    
    df['log'+Y+'_l']= np.log(df[Y+'_l'])
    df['log'+Y+'_u']= np.log(df[Y+'_u'])
    
    return df

In [8]:
# wage_quantiles = np.array(cps['wage'].quantile([0.0, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]))
# wage_quantiles[-1]+=1
# logwage_quantiles = np.log(wage_quantiles)

# cps['wage_qt'] = np.floor(cps['wage'].rank(method = 'max', pct=True)*10).astype(int).replace({10:9})
# cps['wage_lower'] = wage_quantiles[cps['wage_qt']]
# cps['wage_upper'] = wage_quantiles[cps['wage_qt']+1]

# cps['logwage_lower'] = logwage_quantiles[cps['wage_qt']]
# cps['logwage_upper'] = logwage_quantiles[cps['wage_qt']+1]

In [9]:
wage_quantiles = np.array(cps['wage'].quantile(np.linspace(0,1,11)))
wage_quantiles[-1]+=1
cps = createIntervalData(cps, 'wage', wage_quantiles)

In [10]:
cps

,Unnamed: 0,age,wage,educ,log_wage,wage_l,wage_u,logwage_l,logwage_u
0,0,22,12000,13,9.392662,9000.0,15000.0,9.104980,9.615805
1,1,21,3500,13,8.160518,1.0,9000.0,0.000000,9.104980
2,3,49,30000,13,10.308953,30000.0,36000.0,10.308953,10.491274
3,4,31,32000,16,10.373491,30000.0,36000.0,10.308953,10.491274
4,5,42,89630,21,11.403445,71000.0,362303.0,11.170435,12.800236
...,...,...,...,...,...,...,...,...,...
22710,26111,37,32000,14,10.373491,30000.0,36000.0,10.308953,10.491274
22711,26112,50,56000,12,10.933107,53000.0,71000.0,10.878047,11.170435
22712,26113,24,9000,12,9.104980,9000.0,15000.0,9.104980,9.615805
22713,26114,49,29000,12,10.275051,25000.0,30000.0,10.126631,10.308953


# Partial Identification

In [11]:
import setBLP

In [12]:
# Preparing vector and matrix versions of the data

yl = cps.logwage_l;
yu = cps.logwage_u;
x1 = cps.age;
x2 = cps.educ;
x = cps[['age','educ']];

## Testing

In [13]:
setBLP.oneDproj(yl,yu,x1)

[-0.004887858329797926, 0.14865963017398875]

In [14]:
setBLP.oneDproj(yl,yu,x2)

[0.001786042221938278, 0.3685793555816805]

In [15]:
setBLP.oneDproj(yl,yu,x,0)

[-0.009051777215653997, 0.14432389300936818]

In [16]:
setBLP.oneDproj(yl,yu,x,1)

[-0.025567837808786964, 0.3456382284304264]

In [17]:
setBLP.oneDproj(yl,yu,x)

[[-0.009051777215653997, 0.14432389300936818],
 [-0.025567837808786964, 0.3456382284304264]]

In [18]:
setBLP.oneDproj(yl, 'logwage_u', 'age', data = cps)

[-0.004887858329797926, 0.14865963017398875]

In [19]:
setBLP.oneDproj(yl, yu, 'educ', data = cps)

[0.001786042221938278, 0.3685793555816805]

In [20]:
setBLP.oneDproj(yl, yu, ['age','educ'], 0, data = cps)

[-0.009051777215653997, 0.14432389300936818]

In [21]:
setBLP.oneDproj('logwage_l', 'logwage_u', ['age','educ'], 1, data = cps)

[-0.025567837808786964, 0.3456382284304264]

In [22]:
setBLP.oneDproj(yl, yu, ['age','educ'], data = cps)

[[-0.009051777215653997, 0.14432389300936818],
 [-0.025567837808786964, 0.3456382284304264]]

# Testing CI1d  

$H_0: \beta_{age} = [0, 0.15]$

In [23]:
I1 = [0, 0.15]

In [24]:
result = setBLP.oneDproj(yl, yu, x1, H0=I1)

In [25]:
result.bound

[-0.004887858329797926, 0.14865963017398875]

In [26]:
result.Htest.TestStat 

0.7366733861557335

In [27]:
result.Htest.criticalVal 

0.7047452080966871

In [28]:
result.Htest.ConfidenceInterval

[-0.00956387188005852, 0.15333564372424935]

In [29]:
result.dHtest.TestStat 

0.2020137884130314

In [30]:
result.dHtest.criticalVal 

0.5974789676534706

In [31]:
result.dHtest.ConfidenceInterval

[-0.008852155946362981, 0.1526239277905538]

$H_0: \beta_{age} = [0.1, 0.25]$

In [32]:
I2 = [0.1, 0.25]

In [33]:
result = setBLP.oneDproj(yl, yu, x1, H0=I2)

In [34]:
result.bound

[-0.004887858329797926, 0.14865963017398875]

In [35]:
result.Htest.TestStat 

15.808169662239273

In [36]:
result.Htest.criticalVal 

0.7455607394764423

In [37]:
result.Htest.ConfidenceInterval

[-0.009834684615782206, 0.15360645645997303]

In [38]:
result.dHtest.TestStat 

15.273510064496572

In [39]:
result.dHtest.criticalVal 

0.6138535541937418

In [40]:
result.dHtest.ConfidenceInterval

[-0.008960802004062345, 0.15273257384825317]

$H_0: \beta_{age} = [0, 0.1]$

In [41]:
I3 = [0, 0.1]

In [42]:
result = setBLP.oneDproj(yl, yu, x1, H0=I3)

In [43]:
result.bound

[-0.004887858329797926, 0.14865963017398875]

In [44]:
result.Htest.TestStat 

7.333734349628736

In [45]:
result.Htest.criticalVal 

0.7362175670224109

In [46]:
result.Htest.ConfidenceInterval

[-0.009772692280828325, 0.15354446412501915]

In [47]:
result.dHtest.TestStat 

0.0

In [48]:
result.dHtest.criticalVal 

0.620759516254365

In [49]:
result.dHtest.ConfidenceInterval

[-0.009006623347438729, 0.15277839519162956]

$H_0: \beta_{age} = [-0.1, 0.15]$

In [50]:
I4 = [-0.1, 0.15]

In [51]:
result = setBLP.oneDproj(yl, yu, x1, H0=I4)

In [52]:
result.bound

[-0.004887858329797926, 0.14865963017398875]

In [53]:
result.Htest.TestStat 

14.334822889927807

In [54]:
result.Htest.criticalVal 

0.7161447409453018

In [55]:
result.Htest.ConfidenceInterval

[-0.009639508251124772, 0.1534112800953156]

In [56]:
result.dHtest.TestStat 

14.334822889927807

In [57]:
result.dHtest.criticalVal 

0.6004166538992035

In [58]:
result.dHtest.ConfidenceInterval

[-0.008871647615882182, 0.152643419460073]

# regress $yl, yu$ on age, educ $\rightarrow$ $\beta_{age}$

In [65]:
ols3 = smf.ols(formula='logwage_l ~ educ+age',data=cps).fit(cov_type='HC1')
betal = ols3._results.params[2]
print(ols3.summary())

                            OLS Regression Results                            
Dep. Variable:              logwage_l   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     982.3
Date:                Tue, 08 Aug 2023   Prob (F-statistic):               0.00
Time:                        12:11:33   Log-Likelihood:                -56435.
No. Observations:               22715   AIC:                         1.129e+05
Df Residuals:                   22712   BIC:                         1.129e+05
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.3251      0.151     21.988      0.0

In [66]:
ols4 = smf.ols(formula='logwage_u ~ educ+age',data=cps).fit(cov_type='HC1')
betau = ols4._results.params[2]
print(ols4.summary())

                            OLS Regression Results                            
Dep. Variable:              logwage_u   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     3220.
Date:                Tue, 08 Aug 2023   Prob (F-statistic):               0.00
Time:                        12:11:33   Log-Likelihood:                -27984.
No. Observations:               22715   AIC:                         5.597e+04
Df Residuals:                   22712   BIC:                         5.600e+04
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.5141      0.037    201.564      0.0

In [67]:
I5 = [min(betal, betau), max(betal, betau)]

In [68]:
result = setBLP.oneDproj(yl, yu, x1, H0=I5)

In [69]:
result.bound

[-0.004887858329797926, 0.14865963017398875]

In [70]:
result.Htest.TestStat 

7.55663026016013

In [71]:
result.Htest.criticalVal 

0.716500750460006

In [72]:
result.Htest.ConfidenceInterval

[-0.00964187038895225, 0.15341364223314308]

In [73]:
result.dHtest.TestStat 

0.0

In [74]:
result.dHtest.criticalVal 

0.6366335554655311

In [75]:
result.dHtest.ConfidenceInterval

[-0.009111948252945, 0.15288372009713583]

# Simulations

## parameter

In [ ]:
rng = np.random.MT19937(15217)
rng = np.random.Generator(rng)
np.random.seed(setBLP.default_options.seed)

In [ ]:
popSize = cps.shape[0]

In [ ]:
Nobs = 100;  #size of sub sample
Nsim = 5000; #number of simulations

In [ ]:
Nintervals = 8; #number of intervals in the survey

## Using quantiles

In [ ]:
c = 0 
width=0
for i in range(Nsim):
    indx = rng.integers(low=0, high=popSize, size=Nobs)
    sample = cps.iloc[indx, :]
    r = setBLP.oneDproj(sample.logwage_l,sample.logwage_u,sample.educ)
    width += max(r)-min(r)
    if beta1>=min(r) and beta1 <=max(r):
        c+=1
    
width=width/Nsim

In [ ]:
i = 0 
indx = rng.integers(low=0, high=popSize, size=Nobs)
sample = cps.iloc[indx, :]
r = setBLP.oneDproj(sample.logwage_l,sample.logwage_u,sample.educ)

In [ ]:
r

In [ ]:
c/Nsim

In [ ]:
width

## Using fixed intervals, ver 1

In [ ]:
thresholds = np.linspace(1,max(cps.wage)+1,10)
thresholds

In [ ]:
interval_cps = createIntervalData(cps, 'wage', thresholds)
interval_cps.head(15)

In [ ]:
c = 0 
width =0
for i in range(Nsim):
    indx = rng.integers(low=0, high=popSize, size=Nobs)
    sample = interval_cps.iloc[indx, :]
    r = setBLP.oneDproj(sample.logwage_l,sample.logwage_u,sample.educ)
    width+=max(r)-min(r)
    if beta1>=min(r) and beta1 <=max(r):
        c+=1
width=width/Nsim

In [ ]:
c/Nsim

In [ ]:
width

## Using fixed intervals, ver 2

In [ ]:
max(cps.wage)

In [ ]:
thresholds = np.array([0, 10, 20, 40, 75, 100, 200, 300, 500 ])*1000
thresholds[0] =1

In [ ]:
interval_cps = createIntervalData(cps, 'wage', thresholds)
interval_cps.head(15)

In [ ]:
c = 0 
width =0
for i in range(Nsim):
    indx = rng.integers(low=0, high=popSize, size=Nobs)
    sample = interval_cps.iloc[indx, :]
    r = setBLP.oneDproj(sample.logwage_l,sample.logwage_u,sample.educ)
    width+=max(r)-min(r)
    if beta1>=min(r) and beta1 <=max(r):
        c+=1
        
width=width/Nsim

In [ ]:
c/Nsim

In [ ]:
width

### Conclusion:  
The id interval is so wide that there is no chance that true $\beta_1$ is not in the computed identification set.

(age, wage) pair